1.服务器挂载数据集及模块引用

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

In [5]:
import os
os.chdir('/content/drive/MyDrive/') #切换目录，这个是网盘的根目录
os.getcwd()#获取当前目录
os.chdir('商家推荐项目') #切换到data目录

In [ ]:
pip install gensim==4.1.2

In [ ]:
import re
import time
import math
import jieba
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from gensim.models import KeyedVectors, word2vec
from torch.utils.data import DataLoader, WeightedRandomSampler
from torch.utils.data.dataset import TensorDataset
from tqdm import tqdm
from sklearn.metrics import classification_report, accuracy_score

2.数据预处理

In [ ]:
def get_attr_name(attr):
    # 根据索引获取属性名
    a = ['location_traffic_convenience', 'location_distance_from_business_district', 'location_easy_to_find',
         'service_wait_time', 'service_waiters_attitude', 'service_parking_convenience', 'service_serving_speed',
         'price_level', 'price_cost_effective', 'price_discount',
         'environment_decoration', 'environment_noise', 'environment_space', 'environment_cleaness',
         'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
         'others_overall_experience', 'others_willing_to_consume_again'][attr - 2]
    return a

def Get_Label(attr, file_path='./data/train/train.csv'):
    # 将csv文件转化为[属性分类列表]
    label = []
    proportion = [0 for i in range(4)]
    a = get_attr_name(attr)
    raw_data = pd.read_csv(file_path, usecols=[attr])
    safer, pointer = 1, 0  # 内存守护者
    for index, row in raw_data.iterrows():
        # if row[a] == -2: continue
        proportion[row[a]+2] += 1
        # pointer += 1
        # if pointer % safer != 0: continue
        label.append(row[a]+2)
    max_class = max(proportion)
    proportion = [max_class / i for i in proportion]
    return label, proportion
    
def StopwordLoader(file_path='./data/stopwords.txt'):
    stopword = set(line.strip() for line in open(file_path, encoding='UTF-8').readlines())
    return stopword

In [ ]:
#读取文本并分析化
stop_word = StopwordLoader()
def Context_Tokenizer(attr, file_path='./data/train/train.csv', result_path='./trainer.txt'):
    # 将csv文件中的'context'转化为[[分词列表],...]
    sentences = []
    raw_data = pd.read_csv(file_path, usecols=[1, attr])
    a = get_attr_name(attr)
    for index, row in raw_data.iterrows():
        # if row[a] == -2: continue
        content = row['content']
        c_len, poi = len(content), 0
        while poi<c_len:
            if content[poi] == '\n' or content[poi] == ' ' or content[poi] == '\t' or content[poi] == '\r':
                content = content[:poi] + content[poi+1:]
                c_len -= 1
            else: poi += 1
        word_list = [k for k in jieba.cut(content)]
        words = []
        for i in range(len(word_list)):
            if word_list[i] not in stop_word:
                words.append(word_list[i])
        sentences.append(words)
    with open(result_path,'w',encoding='utf-8') as f:
        for sentence in sentences:
            f.write(' '.join(sentence)+'\n')
    return sentences

def Context_Frequency(sentences, file_path='./frequency.txt'):
    # 统计分词结果中的词频，并写入文件
    word_dict = {}
    for sentence in sentences:
        for word in sentence:
            if word not in word_dict.keys():
                word_dict[word] = 1
            else: word_dict[word] += 1
    # word_dict = sorted(list(word_dict.items()),key=lambda x: x[1],reverse=True) #按词频进行排序
    with open(file_path, 'w', encoding='utf-8') as f:
        f.writelines(str(word_dict))

if __name__=='__main__':
    attr = 8
    sentence1 = Context_Tokenizer(attr)
    sentence2 = Context_Tokenizer(attr, file_path='./data/valid/valid.csv', result_path='./valid.txt')
    for item in sentence2:
        sentence1.append(item)
    Context_Frequency(sentence1)

3.模型一(尝试)：基于词典的未提及类别预识别

In [ ]:
#词嵌入向量模型
def read_and_train(file_path='./trainer.txt', model_path='./trainer.model'):
    # 从文本文件中读取分词结果，并提交给word2vec进行训练，并保存word2vec模型
    sentences = []
    with open(file_path,'r',encoding='utf-8') as f:
        while True:
            line = f.readline()
            if len(line) == 0: break
            sentence = line.split()
            sentences.append(sentence)
    model = word2vec.Word2Vec(sentences, workers=10, min_count=0, vector_size=300, alpha=0.025, window=4, negative=4)
    model.save(model_path)

def continue_train(file_path='./valid.txt', model_path='./trainer.model'):
    sentences = []
    with open(file_path, 'r', encoding='utf-8') as f:
        while True:
            line = f.readline()
            if len(line) == 0: break
            sentence = line.split()
            sentences.append(sentence)
    model=word2vec.Word2Vec.load(model_path)
    model.build_vocab(sentences, update=True)
    model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)
    model.save(model_path)

if __name__=='__main__':
    read_and_train()
    continue_train()
    models = word2vec.Word2Vec.load('./trainer.model')
    print(models.wv['好吃'])
    print(models.wv['雌性'])
    print(models.wv['crasyones'])
    print(models.wv['缸内'])

In [ ]:
#统计特定话题的共同高频词表
def oov(attr, file_path='./data/train/train.csv'):
  raw_data = pd.read_csv(file_path, usecols=[1,attr])
  a = get_attr_name(attr)
  mention, oov = dict(), set()
  for index, row in raw_data.iterrows():
        content = row['content']
        c_len, poi = len(content), 0
        while poi<c_len:
            if content[poi] == '\n' or content[poi] == ' ' or content[poi] == '\t' or content[poi] == '\r':
                content = content[:poi] + content[poi+1:]
                c_len -= 1
            else: poi += 1
        word_set = set(k for k in jieba.cut(content))
        if row[a] == -2: oov = oov.union(word_set)
        else:
          for word in word_set:
            if word not in mention.keys(): mention[word]=1
            else: mention[word] += 1
  return mention,oov

if __name__=='__main__':
  attr = 8
  save_name = './' + get_attr_name(attr) + '_mention.txt'
  dic1,oov1 = oov(attr)
  dic2,oov2 = oov(attr, file_path='./data/valid/valid.csv')
  dic = {**dic1,**dic2}
  for word in oov1:
    if word in dic:
      del dic[word]
  for word in oov2:
    if word in dic:
      del dic[word]
  dic = sorted(list(dic.items()),key=lambda y:y[1],reverse=True)
  with open(save_name, 'w', encoding='utf-8') as f:
    for word, fre in dic:
      if fre < 3: break
      f.write(word+' ')

4.模型二(尝试)：基于MLP的未提及类别识别

In [ ]:
#尝试利用线性分类器的方法将-2与{-1,0,1}识别出来
class Mention(nn.Module):
  def __init__(self,vocab_len,batch_size=10,embed_dim=300):
    super(Mention,self).__init__()
    # 词嵌入模型预训练
    embedding_model = word2vec.Word2Vec.load('./trainer.model')
    word2idx = {'None': 0}
    vocab_list = [(k, embedding_model.wv[k]) for k, v in embedding_model.wv.key_to_index.items()]
    embeddings_matrix = np.zeros((len(embedding_model.wv.key_to_index.items()) + 1, embedding_model.vector_size))
    for i in range(len(vocab_list)):
      word = vocab_list[i][0]
      word2idx[word] = i + 1
      embeddings_matrix[i + 1] = vocab_list[i][1]
    embeddings_matrix = torch.tensor(embeddings_matrix, dtype=torch.float)
    # 初始化keras中的Embedding层权重
    self.embedding_table = nn.Embedding(len(embeddings_matrix), embed_dim, _weight= embeddings_matrix)
    self.embedding_table.requires_grad_ = False
    #全连接层
    self.b=batch_size
    self.linear=nn.Linear(60000,300)
    self.linear2=nn.Linear(300,2)
  def forward(self,x):
    x=self.embedding_table(x.long())
    x=torch.reshape(x,(self.b,60000))
    logit2=F.relu(self.linear(x))
    logit=F.relu(self.linear2(logit2))
    return logit

def mention_2_tensor(attr, context_path='./trainer.txt', file_path='./data/train/train.csv',batch_size=10):
  raw_data = pd.read_csv(file_path, usecols=[attr])
  a=get_attr_name(attr)
  labels=[]
  port=[0,0]
  for index, row in raw_data.iterrows():
    if row[a]==-2: 
      labels.append(0)
      port[0]+=1
    else: 
      labels.append(1)
      port[1]+=1
  port=[1/item for item in port]
  vocab = dict()
  vocab['None'] = 0
  embedding_model = word2vec.Word2Vec.load('./trainer.model')
  vocab_list = [k for k,_ in embedding_model.wv.key_to_index.items()]
  for i in range(len(vocab_list)):
    word = vocab_list[i]
    vocab[word] = i + 1
  data = []
  max_len=200
  with open(context_path, 'r', encoding='utf-8') as f:
      pointer = 0
      while True:
          line = f.readline()
          cent = [0 for i in range(max_len)]
          if len(line) == 0: break
          sentence = line.split()
          for index,word in enumerate(sentence):
            if index >= max_len: break
            if word not in vocab.keys():
                vocab[word] = len(vocab)
                cent[index] = vocab[word]
            else:
                cent[index] = vocab[word]
          data.append(cent)
  length=len(labels)
  data = torch.tensor(data, dtype=torch.long)
  labels = torch.tensor(labels, dtype=torch.long)
  print(len(data), len(labels))
  dataset = TensorDataset(data, labels)
  return DataLoader(dataset=dataset, batch_size=batch_size, num_workers=4, shuffle=True, drop_last=True), math.floor(length / batch_size), port, len(vocab)


def accuracy(data_iter, model, batch_count):
    prediction_labels, true_labels = [], []
    with torch.no_grad():
        for count, batch_datas in tqdm(enumerate(data_iter), desc='eval', total=batch_count):
            features, targets = batch_datas
            features, targets = features.cuda(), targets.cuda()
            output = model(features)
            pred = output.softmax(dim=1).argmax(dim=1).cpu()
            prediction_labels.append(pred.numpy())
            true_labels.append(targets.cpu().numpy())
    true_labels = np.concatenate(true_labels)
    prediction_labels = np.concatenate(prediction_labels)
    return classification_report(true_labels, prediction_labels,digits=3)

if __name__=="__main__":
  attr=9
  train_m_dataloader,train_m_batch,p,vocab_len=mention_2_tensor(attr)
  model_m=Mention(vocab_len).cuda()
  test_m_dataloader,test_m_batch,_,_=mention_2_tensor(attr, context_path='./valid.txt', file_path='./data/valid/valid.csv')
  optimizer=torch.optim.Adam(model_m.parameters(),lr=5e-4)
  loss_func=nn.CrossEntropyLoss(torch.tensor(p,dtype=torch.float)).cuda()
  for epoch in range(100):
    start = time.time()
    model_m.train()
    train_loss_sum,train_acc_sum,n=0,0,0
    for step,m_data in tqdm(enumerate(train_m_dataloader),desc='train epoch:{}/{}'.format(epoch + 1, 100), total=train_m_batch):
      optimizer.zero_grad()
      m_train,m_label=m_data
      m_train,m_label=m_train.cuda(),m_label.cuda()
      l=model_m(m_train)
      loss=loss_func(l,m_label)
      loss.backward()
      optimizer.step()
      train_loss_sum+=loss.item()
      logits = l.softmax(dim=1)
      train_acc_sum += (logits.argmax(dim=1) == m_label).sum().item()
      n += m_label.shape[0]
    model_m.eval()
    result = accuracy(test_m_dataloader, model_m, test_m_batch)
    print('epoch %d, loss %.4f, train acc %.3f, time: %.3f' %
    (epoch + 1, train_loss_sum / n, train_acc_sum / n, (time.time() - start)))
    print(result)


5.模型三(对比实验)：基于Text-CNN的细粒度情感分析模型

In [ ]:
#情感分析，将-1,0,1进行分类
class SelfAttention(nn.Module):
    def __init__(self, input_size, hidden_size=300, num_of_header=3, hidden_droput=0.5):
        super(SelfAttention, self).__init__()
        if hidden_size % num_of_header != 0: raise ValueError()
        self.num_of_header = num_of_header
        self.attention_head_size = int(hidden_size/num_of_header)
        self.all_head_size = hidden_size

        self.Q = nn.Linear(input_size, self.all_head_size)
        self.K = nn.Linear(input_size, self.all_head_size)
        self.V = nn.Linear(input_size, self.all_head_size)
        self.attn_dropout = nn.Dropout(hidden_droput)

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_of_header, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(self, input_tensor):
        mixed_Q_layer = self.Q(input_tensor)
        mixed_K_layer = self.K(input_tensor)
        mixed_V_layer = self.V(input_tensor)

        Q_layer = self.transpose_for_scores(mixed_Q_layer)
        K_layer = self.transpose_for_scores(mixed_K_layer)
        V_layer = self.transpose_for_scores(mixed_V_layer)

        attention_scores = torch.matmul(Q_layer, K_layer.transpose(-1,-2))
        attention_scores /= math.sqrt(self.attention_head_size)
        attention_probs = nn.Softmax(dim=-1)(attention_scores)
        attention_probs = self.attn_dropout(attention_probs)

        Context_layer = torch.matmul(attention_probs, V_layer)
        Context_layer = Context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = Context_layer.size()[:-2] + (self.all_head_size,)
        Context_layer = Context_layer.view(*new_context_layer_shape)
        return Context_layer

def text_2_tensor(label, max_len= 200, file_path='./trainer.txt'):
    vocab = dict()
    vocab['None'] = 0
    embedding_model = word2vec.Word2Vec.load('./trainer.model')
    vocab_list = [k for k,_ in embedding_model.wv.key_to_index.items()]
    for i in range(len(vocab_list)):
        word = vocab_list[i]
        vocab[word] = i + 1
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        pointer = 0
        while True:
            # pointer += 1
            line = f.readline()
            cent = [0 for i in range(max_len)]
            if len(line) == 0: break
            # if pointer % 3 != 0: continue
            sentence = line.split()
            for index,word in enumerate(sentence):
                if index >= max_len: break
                if word not in vocab.keys():
                    vocab[word] = len(vocab)
                    cent[index] = vocab[word]
                else:
                    cent[index] = vocab[word]
            data.append(cent)
    length=len(label)
    data = torch.tensor(data, dtype=torch.long)
    label = torch.tensor(label, dtype=torch.long)
    print(len(data), len(label))
    dataset = TensorDataset(data, label)
    return DataLoader(dataset=dataset, batch_size=batch_size, num_workers=4, drop_last=True), math.floor(length / batch_size), len(vocab)


class text_cnn(nn.Module):
    def __init__(self, embed_dim, class_num, kernel_num, kernel_sizes, dropout=0.5):
        super(TextCnn, self).__init__()
        # 词嵌入模型预训练
        embedding_model = word2vec.Word2Vec.load('./trainer.model')
        word2idx = {'None': 0}
        vocab_list = [(k, embedding_model.wv[k]) for k, v in embedding_model.wv.key_to_index.items()]
        embeddings_matrix = np.zeros((len(embedding_model.wv.key_to_index.items()) + 1, embedding_model.vector_size))
        for i in range(len(vocab_list)):
            word = vocab_list[i][0]
            word2idx[word] = i + 1
            embeddings_matrix[i + 1] = vocab_list[i][1]
        embeddings_matrix = torch.tensor(embeddings_matrix, dtype=torch.float)
        # 初始化keras中的Embedding层权重
        self.embedding_table = nn.Embedding(len(embeddings_matrix), embed_dim, _weight= embeddings_matrix)
        self.embedding_table.requires_grad_ = False
        # 注意力机制
        # self.attn = SelfAttention(embed_dim, hidden_size=embed_dim, num_of_header=2, hidden_droput=0.4)
        # text_cnn
        Ci = 1
        Co = kernel_num
        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (f, embed_dim), padding=(2, 0)) for f in kernel_sizes])
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(Co * len(kernel_sizes), class_num)

    def forward(self, x):
        x = self.embedding_table(x.long())
        x = self.attn(x)
        x = x.unsqueeze(1)  # (N, Ci, token_num, embed_dim)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, token_num) * len(kernel_sizes)]
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co) * len(kernel_sizes)]
        x = torch.cat(x, 1) # (N, Co * len(kernel_sizes))
        x = self.dropout(x)  # (N, Co * len(kernel_sizes))
        logit = self.fc(x)  # (N, class_num)
        return logit

# 测试分类结果的准确性
def accuracy(data_iter, model, batch_count):
    prediction_labels, true_labels = [], []
    with torch.no_grad():
        for count, batch_datas in tqdm(enumerate(data_iter), desc='eval', total=batch_count):
            features, targets = batch_datas
            features, targets = features.cuda(), targets.cuda()
            output = model(features)
            pred = output.softmax(dim=1).argmax(dim=1).cpu()
            prediction_labels.append(pred.numpy())
            true_labels.append(targets.cpu().numpy())
    true_labels = np.concatenate(true_labels)
    prediction_labels = np.concatenate(prediction_labels)
    return classification_report(true_labels, prediction_labels)



if __name__ == '__main__':
    # 所需参数
    attr = 17
    batch_size = 10
    embed_dim, class_num, kernel_num, kernel_sizes = 300, 4, 2, [2, 3, 4, 5]
    model = text_cnn(embed_dim, class_num, kernel_num, kernel_sizes).cuda()
    model_name = './' + get_attr_name(attr) + '.pt'
    # 获取训练数据
    train_labels, train_p = Get_Label(attr)
    train_data_loader, train_batch, vocab_len = text_2_tensor(train_labels)
    print(train_p)
    # 获取测试数据
    test_labels, _ = Get_Label(attr, file_path='./data/valid/valid.csv')
    test_data_loader, test_batch, _ = text_2_tensor(test_labels, file_path='./valid.txt')
    # 模型训练与测试
    loss_func = nn.CrossEntropyLoss(torch.tensor(train_p, dtype=torch.float)).cuda()
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-04)
    for epoch in range(100):
        start = time.time()
        model.train()
        train_loss_sum, train_acc_sum, n = 0.0, 0.0, 0
        for step, batch_data in tqdm(enumerate(train_data_loader), desc='train epoch:{}/{}'.format(epoch + 1, 100), total=train_batch):
            feature, target = batch_data
            feature, target = feature.cuda(), target.cuda()
            optimizer.zero_grad()
            logit = model(feature)
            loss = loss_func(logit,target)
            loss.backward()
            optimizer.step()
            train_loss_sum += loss.item()
            logits = logit.softmax(dim=1)
            train_acc_sum += (logits.argmax(dim=1) == target).sum().item()
            n += target.shape[0]
        model.eval()
        result = accuracy(test_data_loader, model, test_batch)
        print('epoch %d, loss %.4f, train acc %.3f, time: %.3f' %
              (epoch + 1, train_loss_sum / n, train_acc_sum / n, (time.time() - start)))
        print(result)
        # if epoch==2: torch.save(model, model_name)


6.模型四：基于Text-CNN+LSTM+多头自注意力机制的细粒度情感分析模型

In [ ]:
from torch.autograd import Variable
#情感分析，将-1,0,1进行分类
def text_2_tensor(label, max_len= 200, file_path='./trainer.txt'):
    vocab = dict()
    vocab['None'] = 0
    embedding_model = word2vec.Word2Vec.load('./trainer.model')
    vocab_list = [k for k,_ in embedding_model.wv.key_to_index.items()]
    for i in range(len(vocab_list)):
        word = vocab_list[i]
        vocab[word] = i + 1
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        pointer = 0
        while True:
            # pointer += 1
            line = f.readline()
            cent = [0 for i in range(max_len)]
            if len(line) == 0: break
            # if pointer % 3 != 0: continue
            sentence = line.split()
            for index,word in enumerate(sentence):
                if index >= max_len: break
                if word not in vocab.keys():
                    vocab[word] = len(vocab)
                    cent[index] = vocab[word]
                else:
                    cent[index] = vocab[word]
            data.append(cent)
    length=len(label)
    data = torch.tensor(data, dtype=torch.long)
    label = torch.tensor(label, dtype=torch.long)
    print(len(data), len(label))
    dataset = TensorDataset(data, label)
    return DataLoader(dataset=dataset, batch_size=batch_size, num_workers=4, drop_last=True), math.floor(length / batch_size), len(vocab)


class TCLA(nn.Module):
    def __init__(self, embed_dim, class_num, kernel_num, kernel_sizes, dropout=0.5):
        super(TextCnn, self).__init__()
        # 词嵌入模型预训练
        embedding_model = word2vec.Word2Vec.load('./trainer.model')
        word2idx = {'None': 0}
        vocab_list = [(k, embedding_model.wv[k]) for k, v in embedding_model.wv.key_to_index.items()]
        embeddings_matrix = np.zeros((len(embedding_model.wv.key_to_index.items()) + 1, embedding_model.vector_size))
        for i in range(len(vocab_list)):
            word = vocab_list[i][0]
            word2idx[word] = i + 1
            embeddings_matrix[i + 1] = vocab_list[i][1]
        embeddings_matrix = torch.tensor(embeddings_matrix, dtype=torch.float)
        # 初始化keras中的Embedding层权重
        self.embedding_table = nn.Embedding(len(embeddings_matrix), embed_dim, _weight= embeddings_matrix)
        self.embedding_table.requires_grad_ = False
        # text_cnn
        Ci = 1
        Co = kernel_num
        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (f, embed_dim), padding=(2, 0)) for f in kernel_sizes])
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(input_size=embed_dim, hidden_size=256, num_layers=1, dropout=0.5, batch_first=True)
        self.lstm_attn = nn.Linear(256, class_num)
        self.word_attn = nn.Linear(200, 1)
        self.fc = nn.Linear(Co * len(kernel_sizes)+class_num, class_num)

    def forward(self, y):
        h0 = Variable(torch.zeros(1, batch_size, 256)).cuda()
        c0 = Variable(torch.zeros(1, batch_size, 256)).cuda()
        y = self.embedding_table(y.long())
        x = y.unsqueeze(1)  # (N, Ci, token_num, embed_dim)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, token_num) * len(kernel_sizes)]
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co) * len(kernel_sizes)]
        x = torch.cat(x, 1) # (N, Co * len(kernel_sizes))
        x = self.dropout(x)  # (N, Co * len(kernel_sizes))
        z,_ = self.lstm(y,(h0,c0))
        z = self.lstm_attn(F.relu(z))
        z = z.permute(0,2,1)
        z = self.word_attn(z).squeeze(2)
        logit = torch.cat((x,z),dim=1)
        logit = self.fc(logit)
        return logit

if __name__ == '__main__':
    # 所需参数
    attr = 17
    batch_size = 10
    embed_dim, class_num, kernel_num, kernel_sizes = 300, 4, 2, [2, 3, 4, 5]
    model = TCLA(embed_dim, class_num, kernel_num, kernel_sizes).cuda()
    model_name = './' + get_attr_name(attr) + '.pt'
    # 获取训练数据
    train_labels, train_p = Get_Label(attr)
    train_data_loader, train_batch, vocab_len = text_2_tensor(train_labels)
    print(train_p)
    # 获取测试数据
    test_labels, _ = Get_Label(attr, file_path='./data/valid/valid.csv')
    test_data_loader, test_batch, _ = text_2_tensor(test_labels, file_path='./valid.txt')
    # 模型训练与测试
    loss_func = nn.CrossEntropyLoss(torch.tensor(train_p, dtype=torch.float)).cuda()
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-04)
    for epoch in range(100):
        start = time.time()
        model.train()
        train_loss_sum, train_acc_sum, n = 0.0, 0.0, 0
        for step, batch_data in tqdm(enumerate(train_data_loader), desc='train epoch:{}/{}'.format(epoch + 1, 100), total=train_batch):
            feature, target = batch_data
            feature, target = feature.cuda(), target.cuda()
            optimizer.zero_grad()
            logit = model(feature)
            loss = loss_func(logit,target)
            loss.backward()
            optimizer.step()
            train_loss_sum += loss.item()
            logits = logit.softmax(dim=1)
            train_acc_sum += (logits.argmax(dim=1) == target).sum().item()
            n += target.shape[0]
        model.eval()
        result = accuracy(test_data_loader, model, test_batch)
        print('epoch %d, loss %.4f, train acc %.3f, time: %.3f' %
              (epoch + 1, train_loss_sum / n, train_acc_sum / n, (time.time() - start)))
        print(result)
        # if epoch==2: torch.save(model, model_name)


7.菜品名称口味聚类模型

In [ ]:
import joblib
import jieba
import re
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

# stopword = StopwordLoader()

def sep_single(name):
    name_depart = jieba.cut(name)
    out = ''
    for word in name_depart:
        # \u4e00-\u9fa5为汉字的正则表达式,^表示对方括号中的正则表达式取补
        if word and not re.match(r'[^A-Z^a-z^0-9^\u4e00-\u9fa5]', word):
            out += word + ' '
    return out

def sep_menu(file_path='./data/menu.csv',labels=['酸','甜','辣','咸','鲜']):
    # 将菜单分割为分词序列
    df = pd.read_csv(file_path, encoding='UTF-8', usecols=[0, 1])
    dish_name, l = [], []
    for index,row in df.iterrows():
        name_depart = sep_single(row['name'])
        # 将标签整型化
        for i in range(len(labels)):
          if row['taste']==labels[i]:
            l.append(i)
            break
        dish_name.append(name_depart)
    return dish_name, l

def count_frequency(corpus, in_file):
    # 获取词频向量，os.path.exists()判断文件是否存在
    if os.path.exists(in_file):
        # 加载保存的模型
        cnt_vector = joblib.load(in_file)
        cnt_tf = cnt_vector.transform(corpus)
    else:
        cnt_vector = CountVectorizer()
        cnt_tf = cnt_vector.fit_transform(corpus)
        print('主题词袋:', len(cnt_vector.get_feature_names()))
        # 保存模型
        joblib.dump(cnt_vector, in_file)
    return cnt_tf

def LDA(in_model, model_in_data, n=2):
    # 训练LDA模型
    if os.path.exists(in_model):
        # 加载已经训练好的模型
        lda = joblib.load(in_model)
        res = lda.transform(model_in_data)
    else:
        # 起初训练一个模型
        lda = LatentDirichletAllocation(n_components=n,
                        # max_iter=5,
                        # learning_method='online',
                        learning_offset=50.,
                        random_state=0)
        res = lda.fit_transform(model_in_data)
        joblib.dump(lda, in_model)
    return res

def K_means(in_model, model_in_data, n=2):
    # 训练LDA模型
    if os.path.exists(in_model):
        # 加载已经训练好的模型
        kmeans = joblib.load(in_model)
        res = kmeans.transform(model_in_data)
    else:
        # 起初训练一个模型
        kmeans = KMeans(n_clusters=n)
        res = kmeans.fit_transform(model_in_data)
        joblib.dump(kmeans, in_model)
    return res

def accuracy(pred, true_labels, file_path='./data/menu.csv'):
    pred_labels = []
    for prediction in pred:
        p = np.argmax(prediction)
        pred_labels.append(p)
    return classification_report(np.concatenate(true_labels), np.concatenate(pred_labels), digit=3)

if __name__ == '__main__':
    # 设定口味类别标签 
    n = len(['酸','甜','辣','咸','鲜'])
    # 获取菜品名称数据和菜品标签
    data_list, dish_label = sep_menu()

    # 训练LDA模型前需要先训练一个CountVectorizer模型
    cv_file = "./CountVectorizer.pkl"
    cnt_data_list = count_frequency(data_list, cv_file)
    lda_model = "./lda_model.pk"
    # 训练LDA模型并计算类别概率
    docres = LDA(lda_model, cnt_data_list, n=n)
    LDA_corpus = np.array(docres)
    # 计算LDA模型的准确率
    # 由于LDA只能确定哪些样本时同一类，不能指示类别标签，所以必须循环修改标签进行比对
    for i in range(n):
        print(accuracy(dish_label, docres))
        for j in range(len(dish_label)):
            dish_label[j] = (dish_label[j] + 1) % n
    # 举例测试LDA预测性能
    test_length = 6
    pre_list = ['杨梅', '香甜奶茶', '咖啡', '辣子鸡', '咖喱鸡', '蒸鱼']
    for i in range(test_length):
        pre_list[i] = sep_single(pre_list[i])
    pre_cnt_data_list = count_frequency(pre_list, cv_file)
    pre_docres = LDA(lda_model, pre_cnt_data_list, n=n)
    print('预测数据概率:\n', np.array(pre_docres))

    # 调用KMeans算法并获取分类结果
    km_model = "./km_model.pk"
    result = K_means(km_model, docres, n=n)
    KM_corpus = np.array(result)
    # 计算KMeans模型的准确率
    # 由于K-means只能确定哪些样本时同一类，不能指示类别标签，所以必须循环修改标签进行比对
    for i in range(n):
        print(accuracy(dish_label, result))
        for j in range(len(dish_label)):
            dish_label[j] = (dish_label[j] + 1) % n
